# IMDB Sentiment Analysis - Complete Pipeline

This notebook provides a complete pipeline for training, testing, and comparing sentiment analysis models on the IMDB reviews dataset.

In [ ]:
%pip install tensorflow tensorflow-datasets scikit-learn matplotlib numpy

## Step 1: Imports and Configuration

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import json
import os
from datetime import datetime
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

# Enable eager execution (required for TensorFlow 2.x)
tf.config.run_functions_eagerly(True) 

# Create model_data directory if it doesn't exist
MODEL_DIR = "model_data"
os.makedirs(MODEL_DIR, exist_ok=True)

print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))
print("Eager execution enabled:", tf.executing_eagerly())

## Step 2: Load Dataset and Visualize Sample

In [ ]:
# Load IMDB reviews dataset
train_ds, test_ds = tfds.load(
    "imdb_reviews",
    split=["train", "test"],
    as_supervised=True,
)

# Display a sample from the dataset
print("=" * 80)
print("SAMPLE REVIEW FROM DATASET")
print("=" * 80)

for text, label in train_ds.take(1):
    sample_text = text.numpy().decode('utf-8')
    sample_label = label.numpy()
    print(f"\nReview Text:\n{sample_text}\n")
    print(f"Label: {sample_label} ({'Positive' if sample_label == 1 else 'Negative'})")
    print("=" * 80)

## Step 3: Create Text Vectorization Layer

In [ ]:
# Hyperparameters for text processing
vocab_size = 10000  # Top 10,000 most frequent words
seq_length = 250    # Maximum review length in words

# Create TextVectorization layer
vectorize_layer = tf.keras.layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=seq_length
)

# Learn vocabulary from training data
train_text = train_ds.map(lambda text, label: text)
vectorize_layer.adapt(train_text)

print(f"Vocabulary size: {vocab_size}")
print(f"Sequence length: {seq_length}")
print(f"Vocabulary learned from {len(list(train_text))} training samples")

## Step 4: Prepare Data Pipeline

In [ ]:
# Preprocessing function
def preprocess_text(text, label):
    text = vectorize_layer(text)
    return text, label

# Batch size
batch_size = 32

# Prepare training dataset with shuffling, batching, and prefetching
train_ds_prepared = (
    train_ds
    .shuffle(10000)
    .map(preprocess_text, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)

# Prepare test dataset (no shuffling needed for testing)
test_ds_prepared = (
    test_ds
    .map(preprocess_text, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)

print(f"Batch size: {batch_size}")
print("Data pipeline prepared successfully!")

## Step 5: Configure Model Name (Set this before training!)

In [ ]:
# ====== CONTROL PARAMETERS - CHANGE THESE BEFORE RUNNING ======

# EXISTING MODEL CONTROLS
USE_EXISTING_MODEL = False  # Set to True to load an existing model, False to create new
EXISTING_MODEL_PATH = "model_data/baseline_model_20251122_151705.h5"  # Path to existing model (start with model_data/)
RETRAIN_EXISTING_MODEL = False  # Set to True to retrain the loaded existing model
RESAVE_EXISTING_MODEL = False  # Set to True to resave the existing model after evaluation

# NEW MODEL CONTROLS
MODEL_NAME = "new_test_model"  # Change this to describe your model if creating new
SAVE_NEW_MODEL = False  # Set to True to save new model to disk, False to keep in memory only

# COMPARISON CONTROLS - Compare current model with another model's metrics
COMPARE_WITH_MODEL = True  # Set to True to enable comparison with another model
COMPARISON_MODEL_JSON = "model_data/baseline_model_20251122_151705.json"  # Path to comparison model's JSON file

# ===============================================================

# Load comparison model data if comparison is enabled
comparison_data = None
if COMPARE_WITH_MODEL:
    if os.path.exists(COMPARISON_MODEL_JSON):
        try:
            with open(COMPARISON_MODEL_JSON, 'r') as f:
                comparison_data = json.load(f)
            print(f"✓ Comparison model loaded: {COMPARISON_MODEL_JSON}")
            print(f"   Model: {comparison_data.get('model_name', 'Unknown')}")
            print(f"   Timestamp: {comparison_data.get('timestamp', 'Unknown')}")
        except Exception as e:
            print(f"⚠ Warning: Could not load comparison model: {e}")
            comparison_data = None
    else:
        print(f"⚠ Warning: Comparison model JSON not found at {COMPARISON_MODEL_JSON}")
        print("   Comparison will be disabled.")
        comparison_data = None

if not USE_EXISTING_MODEL:
    # Generate timestamp for new model
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    model_filename = f"{MODEL_NAME}_{timestamp}"
    model_path = os.path.join(MODEL_DIR, model_filename)

    print(f"\nModel will be created: {model_filename}")
    if SAVE_NEW_MODEL:
        print(f"✓ SAVE_NEW_MODEL = True: Model will be saved to disk")
        print(f"   Save path: {model_path}")
    else:
        print(f"⚠ SAVE_NEW_MODEL = False: Model will be kept in memory only (not saved to disk)")
        print(f"   Note: Model will still be available for comparison using in-memory variables")
else:
    # Validate existing model path
    if os.path.exists(EXISTING_MODEL_PATH):
        print(f"\n✓ Model file found: {EXISTING_MODEL_PATH}")
        
        # For existing models, use the existing path for saving if RESAVE_EXISTING_MODEL is True
        model_path = EXISTING_MODEL_PATH.replace('.h5', '')
        
        # Display flag warnings
        if RETRAIN_EXISTING_MODEL:
            print(f"⚠ RETRAIN_EXISTING_MODEL = True: Model will be retrained")
        if RESAVE_EXISTING_MODEL:
            print(f"⚠ RESAVE_EXISTING_MODEL = True: Model will be resaved after evaluation")
    else:
        print(f"❌ ERROR: Model file not found at {EXISTING_MODEL_PATH}")
        print("Please check the path and try again.")
        raise FileNotFoundError(f"Model file not found: {EXISTING_MODEL_PATH}")

## Step 6: Build or Load Model

In [ ]:
if USE_EXISTING_MODEL:
    # Load existing model (path already validated in Step 5)
    print(f"Loading existing model from: {EXISTING_MODEL_PATH}")
    model = tf.keras.models.load_model(EXISTING_MODEL_PATH)
    print("✓ Model loaded successfully!")
    print(f"Model: {MODEL_NAME}")
    print("Model Architecture:")
    model.summary()
    
    # Try to load corresponding metrics file
    metrics_path = EXISTING_MODEL_PATH.replace('.h5', '.json')
    if os.path.exists(metrics_path):
        with open(metrics_path, 'r') as f:
            model_config = json.load(f)
        print(f"✓ Loaded existing model configuration")
    else:
        # Create minimal config for existing model
        model_config = {
            "model_name": "existing_model",
            "timestamp": datetime.now().strftime("%Y%m%d_%H%M%S"),
            "note": "Loaded from existing file"
        }
        print("⚠ No configuration file found, using minimal config")
else:
    # Build new model
    print(f"Creating new model: {MODEL_NAME}")
    
    # Model hyperparameters
    embedding_dim = 64
    hidden_units = 64
    epochs = 3
    
    # custom glue
    class GELULayer(tf.keras.layers.Layer): # inheriting from Layer class
        def __init__(self, approximate=False, **kwargs): # constructor with custom parameter
            super().__init__(**kwargs) # call the constructor of the parent class
            self.approximate = approximate # store the custom parameter 
        def call(self, inputs): # forward pass inputs is the input tensor to the layer i.e wether the neurons of the layer are activated or not
            return tf.keras.activations.gelu(inputs, approximate=self.approximate) # apply the gelu activation function with the custom parameter to inputs of the layer
        def get_config(self): # method to serialize the layer configuration (so we can save and load the model later)
            cfg = super().get_config()
            cfg.update({"approximate": self.approximate})
            return cfg
    
    # Build the model
    model = tf.keras.Sequential([
        # Embedding layer
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        # Dimensionality reduction
        tf.keras.layers.GlobalAveragePooling1D(),
        # Hidden layers (feedforward)
        tf.keras.layers.Dense(hidden_units, activation=GELULayer(approximate=True)), # using custom GELU activation
        # Output layer
        tf.keras.layers.Dense(1, activation="sigmoid")
    ])
    
    # Compile the model
    model.compile(
        optimizer="adam",
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )
    
    # Display model architecture
    model.build(input_shape=(None, seq_length))
    model.summary()
    
    # Extract model architecture dynamically
    layers_info = []
    for idx, layer in enumerate(model.layers):
        layer_config = layer.get_config()
        layer_info = {
            "layer_type": layer.__class__.__name__,
            "layer_index": idx,
            "layer_name": layer.name,
            "config": layer_config,
            "trainable": layer.trainable
        }
        # Add output shape if available
        try:
            layer_info["output_shape"] = str(layer.output_shape)
        except:
            pass
        layers_info.append(layer_info)
    
    # Extract optimizer configuration
    optimizer_config = model.optimizer.get_config()
    optimizer_name = model.optimizer.__class__.__name__
    
    # Extract loss function
    loss_fn = model.loss
    if hasattr(loss_fn, '__name__'):
        loss_name = loss_fn.__name__
    elif hasattr(loss_fn, 'name'):
        loss_name = loss_fn.name
    else:
        loss_name = str(loss_fn)
    
    # Extract metrics
    metrics_list = []
    for metric in model.metrics:
        if hasattr(metric, 'name'):
            metrics_list.append(metric.name)
        else:
            metrics_list.append(str(metric))
    
    # Store comprehensive model configuration
    model_config = {
        "model_name": MODEL_NAME,
        "timestamp": timestamp,
        "model_type": model.__class__.__name__,
        "hyperparameters": {
            "vocab_size": vocab_size,
            "seq_length": seq_length,
            "embedding_dim": embedding_dim,
            "hidden_units": hidden_units,
            "batch_size": batch_size,
            "epochs": epochs
        },
        "model_architecture": {
            "layers": layers_info,
            "total_layers": len(model.layers),
            "total_params": int(model.count_params()),
            "trainable_params": int(np.sum([int(tf.size(w)) for w in model.trainable_weights])),
            "non_trainable_params": int(np.sum([int(tf.size(w)) for w in model.non_trainable_weights]))
        },
        "compilation": {
            "optimizer": {
                "class_name": optimizer_name,
                "config": optimizer_config
            },
            "loss_function": loss_name,
            "metrics": metrics_list
        }
    }
    print("✓ New model created successfully!")
    
    print("\n Model Configuration:")
    print(json.dumps(model_config, indent=2))

## Step 7: Train the Model

In [ ]:
# Train if creating new model OR if retraining existing model
if not USE_EXISTING_MODEL or (USE_EXISTING_MODEL and RETRAIN_EXISTING_MODEL):
    if USE_EXISTING_MODEL and RETRAIN_EXISTING_MODEL:
        print(f"Retraining existing model: {MODEL_NAME}...")
        # Get epochs from existing model config or use default
        epochs = model_config.get('hyperparameters', {}).get('epochs', 3)
        
        # Recompile model with fresh optimizer to avoid variable binding issues
        print("Recompiling model with fresh optimizer...")
        model.compile(
            optimizer="adam",
            loss="binary_crossentropy",
            metrics=["accuracy"]
        )
    else:
        print(f"Training {MODEL_NAME}...")
    print("=" * 80)
    
    history = model.fit(
        train_ds_prepared,
        epochs=epochs,
        verbose=1 # verbose means show progress bar
    )
    
    # Display comparison with specified comparison model
    if COMPARE_WITH_MODEL and comparison_data is not None:
        comp_history = comparison_data.get("training_history", {})
        if comp_history:
            comp_loss = comp_history.get("loss", [])
            comp_accuracy = comp_history.get("accuracy", [])
            
            print("\n" + "=" * 80)
            print(f"TRAINING COMPARISON: {comparison_data.get('model_name', 'Comparison Model')} vs CURRENT")
            print("=" * 80)
            print(f"\n📊 Comparison Model (Last 3 epochs):")
            print(f"   Loss:     {comp_loss[-3:] if len(comp_loss) >= 3 else comp_loss}")
            print(f"   Accuracy: {comp_accuracy[-3:] if len(comp_accuracy) >= 3 else comp_accuracy}")
            if comp_loss:
                print(f"   Final Loss: {comp_loss[-1]:.4f}")
                print(f"   Final Accuracy: {comp_accuracy[-1]:.4f}")
            
            print(f"\n📊 Current Model:")
            print(f"   Loss:     {[float(f'{val:.4f}') for val in history.history['loss']]}")
            print(f"   Accuracy: {[float(f'{val:.4f}') for val in history.history['accuracy']]}")
            print(f"   Final Loss: {history.history['loss'][-1]:.4f}")
            print(f"   Final Accuracy: {history.history['accuracy'][-1]:.4f}")
            
            if comp_loss:
                print(f"\n📈 Improvement:")
                loss_improvement = comp_loss[-1] - history.history['loss'][-1]
                acc_improvement = history.history['accuracy'][-1] - comp_accuracy[-1]
                print(f"   Loss Change: {loss_improvement:+.4f} {'✓ Better' if loss_improvement > 0 else '✗ Worse'}")
                print(f"   Accuracy Change: {acc_improvement:+.4f} {'✓ Better' if acc_improvement > 0 else '✗ Worse'}")
    # Display comparison if retraining existing model (backward compatibility)
    elif "training_history" in model_config and USE_EXISTING_MODEL and RETRAIN_EXISTING_MODEL:
        old_loss = model_config["training_history"].get("loss", [])
        old_accuracy = model_config["training_history"].get("accuracy", [])
        
        print("\n" + "=" * 80)
        print("TRAINING COMPARISON: OLD vs NEW")
        print("=" * 80)
        print(f"\n📊 Previous Training (Last 3 epochs):")
        print(f"   Loss:     {old_loss[-3:] if len(old_loss) >= 3 else old_loss}")
        print(f"   Accuracy: {old_accuracy[-3:] if len(old_accuracy) >= 3 else old_accuracy}")
        print(f"   Final Loss: {old_loss[-1]:.4f}")
        print(f"   Final Accuracy: {old_accuracy[-1]:.4f}")
        
        print(f"\n📊 Current Retraining:")
        print(f"   Loss:     {[float(f'{val:.4f}') for val in history.history['loss']]}")
        print(f"   Accuracy: {[float(f'{val:.4f}') for val in history.history['accuracy']]}")
        print(f"   Final Loss: {history.history['loss'][-1]:.4f}")
        print(f"   Final Accuracy: {history.history['accuracy'][-1]:.4f}")
        
        print(f"\n📈 Improvement:")
        loss_improvement = old_loss[-1] - history.history['loss'][-1]
        acc_improvement = history.history['accuracy'][-1] - old_accuracy[-1]
        print(f"   Loss Change: {loss_improvement:+.4f} {'✓ Better' if loss_improvement > 0 else '✗ Worse'}")
        print(f"   Accuracy Change: {acc_improvement:+.4f} {'✓ Better' if acc_improvement > 0 else '✗ Worse'}")
    
    # Store only current training history (overwrite, don't append)
    model_config["training_history"] = {
        "loss": [float(val) for val in history.history['loss']],
        "accuracy": [float(val) for val in history.history['accuracy']],
        "epochs_completed": len(history.history['loss'])
    }
    
    print("\n" + "=" * 80)
    print("Training complete!")
    print("=" * 80)
else:
    print("=" * 80)
    print("SKIPPED TRAINING (Using existing model)")
    print("=" * 80)
    

## Step 8: Evaluate Model on Test Set

In [ ]:
# Evaluate on test set
print("Evaluating model on test set...")
test_loss, test_accuracy = model.evaluate(test_ds_prepared, verbose=1)

# Get predictions for all test data
print("\nGenerating predictions...")
y_true = []
y_pred_probs = []

for texts, labels in test_ds_prepared:
    predictions = model.predict(texts, verbose=0)
    y_true.extend(labels.numpy())
    y_pred_probs.extend(predictions.flatten())

y_true = np.array(y_true)
y_pred_probs = np.array(y_pred_probs)
y_pred = (y_pred_probs > 0.5).astype(int)

# Calculate comprehensive metrics
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
roc_auc = roc_auc_score(y_true, y_pred_probs)

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)

print("\n" + "=" * 80)
print("EVALUATION METRICS")
print("=" * 80)
print(f"Test Loss:      {test_loss:.4f}")
print(f"Test Accuracy:  {test_accuracy:.4f}")
print(f"Precision:      {precision:.4f}")
print(f"Recall:         {recall:.4f}")
print(f"F1 Score:       {f1:.4f}")
print(f"ROC AUC Score:  {roc_auc:.4f}")
print("\nConfusion Matrix:")
print(f"                Predicted Negative  Predicted Positive")
print(f"Actual Negative        {cm[0,0]:5d}              {cm[0,1]:5d}")
print(f"Actual Positive        {cm[1,0]:5d}              {cm[1,1]:5d}")
print("=" * 80)

# Calculate additional metrics
from sklearn.metrics import accuracy_score, matthews_corrcoef

# True/False Positives/Negatives
tn, fp, fn, tp = cm[0, 0], cm[0, 1], cm[1, 0], cm[1, 1]
total_predictions = tn + fp + fn + tp

# Specificity (True Negative Rate)
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

# False Positive Rate
fpr = fp / (fp + tn) if (fp + tn) > 0 else 0

# False Negative Rate
fnr = fn / (fn + tp) if (fn + tp) > 0 else 0

# Matthews Correlation Coefficient
mcc = matthews_corrcoef(y_true, y_pred)

# Display comparison with specified comparison model
if COMPARE_WITH_MODEL and comparison_data is not None:
    comp_metrics = comparison_data.get("evaluation_metrics", {})
    if comp_metrics:
        print("\n" + "=" * 80)
        print(f"EVALUATION COMPARISON: {comparison_data.get('model_name', 'Comparison Model')} vs CURRENT")
        print("=" * 80)
        
        print(f"\n{'Metric':<20} {'Comparison':<12} {'Current':<12} {'Change':<12} {'Status'}")
        print("-" * 80)
        
        metrics_to_compare = [
            ("Test Loss", comp_metrics.get('test_loss', 0), test_loss, False),
            ("Test Accuracy", comp_metrics.get('test_accuracy', 0), test_accuracy, True),
            ("Precision", comp_metrics.get('precision', 0), precision, True),
            ("Recall", comp_metrics.get('recall', 0), recall, True),
            ("F1 Score", comp_metrics.get('f1_score', 0), f1, True),
            ("ROC AUC", comp_metrics.get('roc_auc_score', 0), roc_auc, True),
        ]
        
        for metric_name, comp_val, curr_val, higher_is_better in metrics_to_compare:
            change = curr_val - comp_val
            if higher_is_better:
                status = "✓ Better" if change > 0 else ("✗ Worse" if change < 0 else "= Same")
            else:
                status = "✓ Better" if change < 0 else ("✗ Worse" if change > 0 else "= Same")
            print(f"{metric_name:<20} {comp_val:<12.4f} {curr_val:<12.4f} {change:+12.4f} {status}")
        
        print("=" * 80)
# Display comparison if retraining existing model (backward compatibility)
elif "evaluation_metrics" in model_config and USE_EXISTING_MODEL and RETRAIN_EXISTING_MODEL:
    old_metrics = model_config["evaluation_metrics"]
    
    print("\n" + "=" * 80)
    print("EVALUATION COMPARISON: OLD vs NEW")
    print("=" * 80)
    
    print(f"\n{'Metric':<20} {'Old':<12} {'New':<12} {'Change':<12} {'Status'}")
    print("-" * 80)
    
    metrics_to_compare = [
        ("Test Loss", old_metrics.get('test_loss', 0), test_loss, False),
        ("Test Accuracy", old_metrics.get('test_accuracy', 0), test_accuracy, True),
        ("Precision", old_metrics.get('precision', 0), precision, True),
        ("Recall", old_metrics.get('recall', 0), recall, True),
        ("F1 Score", old_metrics.get('f1_score', 0), f1, True),
        ("ROC AUC", old_metrics.get('roc_auc_score', 0), roc_auc, True),
    ]
    
    for metric_name, old_val, new_val, higher_is_better in metrics_to_compare:
        change = new_val - old_val
        if higher_is_better:
            status = "✓ Better" if change > 0 else ("✗ Worse" if change < 0 else "= Same")
        else:
            status = "✓ Better" if change < 0 else ("✗ Worse" if change > 0 else "= Same")
        print(f"{metric_name:<20} {old_val:<12.4f} {new_val:<12.4f} {change:+12.4f} {status}")
    
    print("=" * 80)

# Store comprehensive metrics in config
model_config["evaluation_metrics"] = {
    "test_loss": float(test_loss),
    "test_accuracy": float(test_accuracy),
    "precision": float(precision),
    "recall": float(recall),
    "f1_score": float(f1),
    "roc_auc_score": float(roc_auc),
    "specificity": float(specificity),
    "matthews_corrcoef": float(mcc),
    "confusion_matrix": {
        "matrix": cm.tolist(),
        "true_negatives": int(tn),
        "false_positives": int(fp),
        "false_negatives": int(fn),
        "true_positives": int(tp)
    },
    "error_rates": {
        "false_positive_rate": float(fpr),
        "false_negative_rate": float(fnr)
    },
    "total_predictions": int(total_predictions),
    "correct_predictions": int(tp + tn),
    "incorrect_predictions": int(fp + fn)
}

## Step 9: Test Single Example (Detailed Prediction)

In [ ]:
# Get one example from test set for detailed analysis
print("=" * 80)
print("SINGLE TEST EXAMPLE - DETAILED PREDICTION")
print("=" * 80)

for text, label in test_ds.take(1):
    # Get the original text
    original_text = text.numpy().decode('utf-8')
    true_label = label.numpy()
    
    # Preprocess and predict
    preprocessed = vectorize_layer(tf.expand_dims(text, 0))
    prediction_prob = model.predict(preprocessed, verbose=0)[0][0]
    predicted_label = 1 if prediction_prob > 0.5 else 0
    
    # Display results
    print(f"\nReview Text:\n{original_text}\n")
    print("-" * 80)
    print(f"True Label:           {true_label} ({'Positive' if true_label == 1 else 'Negative'})")
    print(f"Predicted Label:      {predicted_label} ({'Positive' if predicted_label == 1 else 'Negative'})")
    print(f"Prediction Confidence: {prediction_prob:.4f} ({prediction_prob*100:.2f}%)")
    print(f"Correct Prediction:   {'✓ YES' if predicted_label == true_label else '✗ NO'}")
    print("=" * 80)

## Step 10: Save Model and Metrics

In [ ]:
# Save if: (creating new model AND SAVE_NEW_MODEL is True) OR (using existing AND RESAVE_EXISTING_MODEL is True)
if (not USE_EXISTING_MODEL and SAVE_NEW_MODEL) or (USE_EXISTING_MODEL and RESAVE_EXISTING_MODEL):
    # Save the model
    model_file = f"{model_path}.h5"
    model.save(model_file)
    
    if USE_EXISTING_MODEL and RESAVE_EXISTING_MODEL:
        print(f"✓ Existing model updated and saved to: {model_file}")
    else:
        print(f"✓ Model saved to: {model_file}")
    
    # Save the metrics and configuration
    metrics_file = f"{model_path}.json"
    with open(metrics_file, 'w') as f:
        json.dump(model_config, f, indent=4)
    
    if USE_EXISTING_MODEL and RESAVE_EXISTING_MODEL:
        print(f"✓ Existing model configuration updated and saved to: {metrics_file}")
    else:
        print(f"✓ Metrics saved to: {metrics_file}")
    
    print("\n" + "=" * 80)
    print("MODEL AND METRICS SAVED SUCCESSFULLY!")
    print("=" * 80)
else:
    print("\n" + "=" * 80)
    if USE_EXISTING_MODEL:
        print("SKIPPED SAVING (Using existing model without resave)")
    else:
        print("SKIPPED SAVING (SAVE_NEW_MODEL = False, model kept in memory only)")
    print("=" * 80)

---
## MODEL COMPARISON AND RANKING SYSTEM

Run the cells below to compare all trained models and see rankings.

## Step 11: Load and Rank All Models

In [ ]:
import glob

# Load all model metrics from the model_data directory
def load_all_models():
    """Load metrics for all trained models (from disk and memory)"""
    json_files = glob.glob(os.path.join(MODEL_DIR, "*.json"))
    
    models_data = []
    
    # Load models from disk
    for json_file in json_files:
        try:
            with open(json_file, 'r') as f:
                data = json.load(f)
                # Add filename for reference
                data['filename'] = os.path.basename(json_file).replace('.json', '')
                data['source'] = 'disk'
                models_data.append(data)
        except Exception as e:
            print(f"Error loading {json_file}: {e}")
    
    # Check if there's an in-memory model (model_config exists and not saved to disk)
    if 'model_config' in globals():
        config = globals()['model_config']
        # Check if this model was saved to disk by looking for matching timestamp
        is_saved = any(m.get('timestamp') == config.get('timestamp') for m in models_data)
        
        if not is_saved:
            # Add in-memory model to the list
            in_memory_model = config.copy()
            in_memory_model['filename'] = f"{config.get('model_name', 'in_memory')}_{config.get('timestamp', 'unknown')}"
            in_memory_model['source'] = 'memory'
            models_data.append(in_memory_model)
            print("✓ Included in-memory model in comparison")
    
    if not models_data:
        print("No trained models found (disk or memory).")
        print("Train a model first by running the cells above!")
        return []
    
    return models_data

# Load all models
all_models = load_all_models()
print(f"Found {len(all_models)} trained model(s)")

## Step 12: Rank Models by Performance

In [ ]:
def calculate_ranking_score(model_data):
    """
    Calculate a composite ranking score based on multiple metrics.
    Higher is better. Weighted combination of:
    - F1 Score (40%)
    - ROC AUC (30%)
    - Accuracy (20%)
    - Precision (10%)
    """
    # Support both old and new metric format
    metrics = model_data.get('evaluation_metrics', model_data.get('metrics', {}))
    
    f1 = metrics.get('f1_score', 0)
    roc_auc = metrics.get('roc_auc_score', metrics.get('roc_auc', 0))
    accuracy = metrics.get('test_accuracy', 0)
    precision = metrics.get('precision', 0)
    
    # Weighted score
    score = (f1 * 0.4) + (roc_auc * 0.3) + (accuracy * 0.2) + (precision * 0.1)
    return score

# Rank models
if all_models:
    for model in all_models:
        model['ranking_score'] = calculate_ranking_score(model)
    
    # Sort by ranking score (best first)
    ranked_models = sorted(all_models, key=lambda x: x['ranking_score'], reverse=True)
    
    # Display rankings
    print("=" * 100)
    print("MODEL RANKINGS (Best to Worst)")
    print("=" * 100)
    print(f"{'Rank':<6} {'Model Name':<35} {'F1':<8} {'ROC AUC':<8} {'Accuracy':<8} {'Score':<8} {'Source':<8}")
    print("-" * 100)
    
    for rank, model in enumerate(ranked_models, 1):
        name = model['model_name']
        timestamp = model['timestamp']
        metrics = model.get('evaluation_metrics', model.get('metrics', {}))
        source = model.get('source', 'disk')
        
        f1 = metrics.get('f1_score', 0)
        roc_auc = metrics.get('roc_auc_score', metrics.get('roc_auc', 0))
        accuracy = metrics.get('test_accuracy', 0)
        score = model['ranking_score']
        
        display_name = f"{name} ({timestamp})"
        source_indicator = "💾 Disk" if source == 'disk' else "🧠 Memory"
        print(f"{rank:<6} {display_name:<35} {f1:<8.4f} {roc_auc:<8.4f} {accuracy:<8.4f} {score:<8.4f} {source_indicator:<8}")
    
    print("=" * 100)
else:
    print("No models to rank. Train some models first!")

## Step 13: Display Best Model in Detail

In [ ]:
if all_models and ranked_models:
    best_model = ranked_models[0]
    
    print("\n" + "=" * 100)
    print("🏆 BEST MODEL - DETAILED INFORMATION")
    print("=" * 100)
    
    print(f"\n📋 MODEL IDENTIFICATION")
    print(f"   Name:           {best_model['model_name']}")
    print(f"   Timestamp:      {best_model['timestamp']}")
    print(f"   Filename:       {best_model['filename']}")
    print(f"   Source:         {'💾 Saved to Disk' if best_model.get('source', 'disk') == 'disk' else '🧠 In-Memory Only'}")
    print(f"   Ranking Score:  {best_model['ranking_score']:.6f}")
    
    print(f"\n🏗️  MODEL ARCHITECTURE")
    arch = best_model.get('model_architecture', best_model.get('architecture', {}))
    hyperparams = best_model.get('hyperparameters', {})
    print(f"   Vocabulary Size:    {hyperparams.get('vocab_size', arch.get('vocab_size', 'N/A'))}")
    print(f"   Sequence Length:    {hyperparams.get('seq_length', arch.get('seq_length', 'N/A'))}")
    print(f"   Embedding Dim:      {hyperparams.get('embedding_dim', arch.get('embedding_dim', 'N/A'))}")
    print(f"   Hidden Units:       {hyperparams.get('hidden_units', arch.get('hidden_units', 'N/A'))}")
    
    compilation = best_model.get('compilation', {})
    print(f"   Optimizer:          {compilation.get('optimizer', arch.get('optimizer', 'N/A'))}")
    print(f"   Loss Function:      {compilation.get('loss_function', arch.get('loss', 'N/A'))}")
    
    print(f"\n⚙️  TRAINING PARAMETERS")
    print(f"   Batch Size:         {hyperparams.get('batch_size', 'N/A')}")
    print(f"   Epochs:             {hyperparams.get('epochs', 'N/A')}")
    
    print(f"\n📊 PERFORMANCE METRICS")
    metrics = best_model.get('evaluation_metrics', best_model.get('metrics', {}))
    print(f"   Test Accuracy:      {metrics.get('test_accuracy', 0):.4f} ({metrics.get('test_accuracy', 0)*100:.2f}%)")
    print(f"   Test Loss:          {metrics.get('test_loss', 0):.4f}")
    print(f"   Precision:          {metrics.get('precision', 0):.4f}")
    print(f"   Recall:             {metrics.get('recall', 0):.4f}")
    print(f"   F1 Score:           {metrics.get('f1_score', 0):.4f}")
    print(f"   ROC AUC Score:      {metrics.get('roc_auc_score', metrics.get('roc_auc', 0)):.4f}")
    
    print(f"\n📈 CONFUSION MATRIX")
    cm_data = metrics.get('confusion_matrix', {})
    cm = cm_data.get('matrix', [[0, 0], [0, 0]]) if isinstance(cm_data, dict) else cm_data
    print(f"                      Predicted Negative    Predicted Positive")
    print(f"   Actual Negative          {cm[0][0]:6d}                {cm[0][1]:6d}")
    print(f"   Actual Positive          {cm[1][0]:6d}                {cm[1][1]:6d}")
    
    # Calculate additional insights
    total = sum(sum(row) for row in cm)
    tn, fp, fn, tp = cm[0][0], cm[0][1], cm[1][0], cm[1][1]
    
    print(f"\n💡 ADDITIONAL INSIGHTS")
    print(f"   True Negatives:     {tn:6d} ({tn/total*100:.2f}%)")
    print(f"   False Positives:    {fp:6d} ({fp/total*100:.2f}%)")
    print(f"   False Negatives:    {fn:6d} ({fn/total*100:.2f}%)")
    print(f"   True Positives:     {tp:6d} ({tp/total*100:.2f}%)")
    print(f"   Total Predictions:  {total:6d}")
    
    print("\n" + "=" * 100)
else:
    print("No models available for detailed display.")

## Step 14: Visualize Model Comparison

In [ ]:
if all_models and len(ranked_models) > 0:
    # Prepare data for visualization
    model_names = [f"{m['model_name']}\n{m['timestamp']}" for m in ranked_models]
    # Support both old and new format
    accuracies = [m.get('evaluation_metrics', m.get('metrics', {})).get('test_accuracy', 0) for m in ranked_models]
    f1_scores = [m.get('evaluation_metrics', m.get('metrics', {})).get('f1_score', 0) for m in ranked_models]
    roc_aucs = [m.get('evaluation_metrics', m.get('metrics', {})).get('roc_auc_score', m.get('evaluation_metrics', m.get('metrics', {})).get('roc_auc', 0)) for m in ranked_models]
    precisions = [m.get('evaluation_metrics', m.get('metrics', {})).get('precision', 0) for m in ranked_models]
    recalls = [m.get('evaluation_metrics', m.get('metrics', {})).get('recall', 0) for m in ranked_models]
    
    # Create comparison plot
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Model Performance Comparison', fontsize=16, fontweight='bold')
    
    # Plot 1: Accuracy comparison
    axes[0, 0].barh(model_names, accuracies, color='steelblue')
    axes[0, 0].set_xlabel('Accuracy')
    axes[0, 0].set_title('Test Accuracy')
    axes[0, 0].set_xlim(0, 1)
    for i, v in enumerate(accuracies):
        axes[0, 0].text(v + 0.01, i, f'{v:.4f}', va='center')
    
    # Plot 2: F1 Score comparison
    axes[0, 1].barh(model_names, f1_scores, color='coral')
    axes[0, 1].set_xlabel('F1 Score')
    axes[0, 1].set_title('F1 Score')
    axes[0, 1].set_xlim(0, 1)
    for i, v in enumerate(f1_scores):
        axes[0, 1].text(v + 0.01, i, f'{v:.4f}', va='center')
    
    # Plot 3: ROC AUC comparison
    axes[1, 0].barh(model_names, roc_aucs, color='mediumseagreen')
    axes[1, 0].set_xlabel('ROC AUC')
    axes[1, 0].set_title('ROC AUC Score')
    axes[1, 0].set_xlim(0, 1)
    for i, v in enumerate(roc_aucs):
        axes[1, 0].text(v + 0.01, i, f'{v:.4f}', va='center')
    
    # Plot 4: Precision vs Recall
    axes[1, 1].scatter(recalls, precisions, s=200, alpha=0.6, c=range(len(model_names)), cmap='viridis')
    for i, name in enumerate(model_names):
        axes[1, 1].annotate(f'#{i+1}', (recalls[i], precisions[i]), 
                           ha='center', va='center', fontweight='bold', color='white')
    axes[1, 1].set_xlabel('Recall')
    axes[1, 1].set_ylabel('Precision')
    axes[1, 1].set_title('Precision vs Recall')
    axes[1, 1].set_xlim(0, 1)
    axes[1, 1].set_ylim(0, 1)
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("✓ Visualization complete!")
else:
    print("Need at least one model to visualize. Train some models first!")